In [1]:
import enum
import importlib
import inspect
import os
import shutil

from typing import List
from IPython.display import HTML
from sflkit.analysis.spectra import Line

from sflkit.analysis import analysis_type, factory
from sflkit import *
from sflkit.color import ColorCode
from sflkit import instrument_config, analyze_config
from sflkit.config import Config

from avicenna import *

## A faulty Program

First, we need a faulty program. We chose an implementation of the `middle(x, y, z)` function that returns the *middle* number of its three arguments. For example, `middle(1, 3, 2)` should return 2 because `1 < 2` and `2 < 3`. We introduced a fault in this implementation of `middle` that occurs in line 7 `m = y`. 

In [2]:
def middle(x, y, z):
    m = z
    if y < z:
        if x < y:
            m = y
        elif x < z:
            m = y 
    else:
        if x > y:
            m = y
        elif x > z:
            m = x # desired line
    return m

In [3]:
import ast
import os
import random
import string
import subprocess
from _ast import Call, ImportFrom
from pathlib import Path
from typing import List, Optional, Tuple, Any, Callable

from tests4py.constants import PYTHON
from tests4py.grammars import python
from tests4py.grammars.fuzzer import Grammar
from tests4py.grammars.fuzzer import is_valid_grammar
from tests4py.grammars.fuzzer import srange
from tests4py.projects import Project, Status, TestingFramework, TestStatus
from tests4py.tests.generator import UnittestGenerator, SystemtestGenerator
from tests4py.tests.utils import API, TestResult

PROJECT_MAME = "expression"


class Expression(Project):
    def __init__(
        self,
        bug_id: int,
        buggy_commit_id: str,
        fixed_commit_id: str,
        test_files: List[Path],
        test_cases: List[str],
        test_status_fixed: TestStatus = TestStatus.PASSING,
        test_status_buggy: TestStatus = TestStatus.FAILING,
        unittests: Optional[UnittestGenerator] = None,
        systemtests: Optional[SystemtestGenerator] = None,
        api: Optional[API] = None,
        loc: int = 0,
    ):
        super().__init__(
            bug_id=bug_id,
            project_name=PROJECT_MAME,
            github_url="https://github.com/smythi93/expression",
            status=Status.OK,
            python_version="3.10.9",
            python_path="",
            buggy_commit_id=buggy_commit_id,
            fixed_commit_id=fixed_commit_id,
            testing_framework=TestingFramework.PYTEST,
            test_files=test_files,
            test_cases=test_cases,
            test_status_fixed=test_status_fixed,
            test_status_buggy=test_status_buggy,
            unittests=unittests,
            systemtests=systemtests,
            api=api,
            grammar=grammar,
            loc=loc,
            setup=[[PYTHON, "-m", "pip", "install", "-e", "."]],
            included_files=[os.path.join("src", PROJECT_MAME)],
            test_base=Path("tests"),
        )


def register():
    Expression(
        bug_id=1,
        buggy_commit_id="7b08a1dd737bd47c9be47258d2cd63bb7de72c47",
        fixed_commit_id="10356a6d3768db2ff7749408b7f3d12a773a18a9",
        test_files=[
            Path("tests", "test_evaluate.py"),
            Path("tests", "test_expression.py"),
        ],
        test_cases=[
            os.path.join("tests", "test_evaluate.py")
            + "::TestEvaluate::test_eval_div_error",
            os.path.join("tests", "test_expression.py") + "::TestExpr::test_div_error",
        ],
        loc=108,
        unittests=ExpressionUnittestGenerator(),
        systemtests=ExpressionSystemtestGenerator(),
        api=ExpressionAPI(),
    )


class ExpressionAPI(API):
    def __init__(self, default_timeout=5):
        super().__init__(default_timeout=default_timeout)

    def get_test_arguments_from_string(self, s: str) -> List[str]:
        return [s]

    def oracle(self, args: Any) -> Tuple[TestResult, str]:
        if args is None:
            return TestResult.UNDEFINED, "No process finished"
        process: subprocess.CompletedProcess = args
        if process.returncode != 0 and b"ValueError" not in process.stderr:
            return TestResult.FAILING, f"Process failed with {process.returncode}"
        elif process.returncode != 0:
            return (
                TestResult.PASSING,
                f"Process failed with ValueError and code {process.returncode}",
            )
        expected = process.args[2]
        expected = "".join(expected).strip()
        result = process.stdout.decode("utf8")
        result = float(result)
        try:
            expected = eval(expected.replace("~", "-"))
        except ZeroDivisionError:
            return (
                TestResult.FAILING,
                "ZeroDivisionError for evaluation but no ValueError for subject.",
            )
        if result == expected:
            return TestResult.PASSING, f"Expected {expected}, and was {result}"
        else:
            return TestResult.FAILING, f"Expected {expected}, but was {result}"


class ExpressionTestGenerator:
    @staticmethod
    def generate_values(producer: Callable) -> Tuple[Any, Any]:
        return tuple(producer())

    @staticmethod
    def _generate_int() -> int:
        value = random.randint(1, 1001)
        return value

    @staticmethod
    def _generate_symbol() -> str:
        symbols_ = ["+", "-", "/", "*"]
        return symbols_[random.randint(0, 3)]

    def generate_structure_(self):
        structure_ = (
            f"{self._generate_int()}",
            f"{self._generate_int()} {self._generate_symbol()} {self._generate_int()}",
        )
        structure_zero_div_ = " " + "/" + " " + "0" + " "
        value_for_sub = self._generate_int()
        structure_sub_zero_div_ = (
            "(" + str(value_for_sub) + " - " + str(value_for_sub) + ")"
        )
        structure_passing_ = (
            "("
            + (
                structure_[1]
                + ")"
                + " "
                + self._generate_symbol()
                + " "
                + structure_[0]
            ),
            (structure_[1]),
            ("(" + structure_[1] + ")"),
            (
                structure_[0]
                + " "
                + self._generate_symbol()
                + " "
                + "("
                + structure_[1]
                + ")"
            ),
        )
        structure_failing_ = (
            ("(" + structure_[1] + ")" + structure_zero_div_),
            (
                structure_[0]
                + structure_zero_div_
                + self._generate_symbol()
                + " "
                + structure_[0]
            ),
            (structure_[0] + structure_zero_div_),
            (structure_[0] + " / " + structure_sub_zero_div_),
        )
        return (
            structure_passing_[random.randint(0, 3)],
            structure_failing_[random.randint(0, 3)],
        )


class ExpressionUnittestGenerator(
    python.PythonGenerator, UnittestGenerator, ExpressionTestGenerator
):
    def _generate_one(
        self,
    ) -> tuple:
        return self.generate_values(self.generate_structure_)

    @staticmethod
    def _get_assert(
        expected: str,
        result: str,
    ) -> list[Call]:
        return [
            ast.Call(
                func=ast.Attribute(value=ast.Name(id="self"), attr="assertEqual"),
                args=[
                    ast.Constant(value=expected),
                    ast.Call(
                        func=ast.Name(id="evaluate"),
                        args=(
                            [
                                ast.Constant(value=result),
                            ],
                        ),
                        keywords=[],
                    ),
                ],
                keywords=[],
            ),
        ]

    @staticmethod
    def _get_assert_with_error_(
        expected: ZeroDivisionError | str,
        result: str,
    ) -> list[Call]:
        return [
            ast.Call(
                func=ast.Attribute(value=ast.Name(id="self"), attr="assertRaises"),
                args=[
                    ast.Constant(value=expected),
                    ast.Call(
                        func=ast.Name(id="evaluate"),
                        args=(
                            [
                                ast.Constant(value=result),
                            ],
                        ),
                        keywords=[],
                    ),
                ],
                keywords=[],
            ),
        ]

    def get_imports(self) -> list[ImportFrom]:
        return [
            ast.ImportFrom(
                module="expression.evaluate",
                names=[
                    ast.alias(name="evaluate"),
                ],
                level=0,
            ),
            ast.ImportFrom(
                module="expression.expr.arithmetic",
                names=[
                    ast.alias(name="Constant"),
                    ast.alias(name="Div"),
                    ast.alias(name="Add"),
                    ast.alias(name="Mul"),
                ],
                level=0,
            ),
            ast.ImportFrom(
                module="expression.expr.parse",
                names=[
                    ast.alias(name="parse"),
                ],
                level=0,
            ),
        ]

    def generate_failing_test(self) -> Tuple[ast.FunctionDef, TestResult]:
        _, generated_value = self._generate_one()
        print("fail :", generated_value)
        test = self.get_empty_test()
        test.body = self._get_assert_with_error_("ZeroDivisionError", generated_value)
        return test, TestResult.FAILING

    def generate_passing_test(self) -> Tuple[ast.FunctionDef, TestResult]:
        generated_value, _ = self._generate_one()
        print("pass :", generated_value)
        test = self.get_empty_test()
        test.body = self._get_assert(eval(generated_value), generated_value)
        return test, TestResult.PASSING


class ExpressionSystemtestGenerator(SystemtestGenerator, ExpressionTestGenerator):
    def generate_failing_test(self) -> Tuple[str, TestResult]:
        _, generated_value = self.generate_values(self.generate_structure_)
        return f"{generated_value}", TestResult.FAILING

    def generate_passing_test(self) -> Tuple[str, TestResult]:
        generated_value, _ = self.generate_values(self.generate_structure_)
        return f"{generated_value}", TestResult.PASSING


grammar: Grammar = {
    "<start>": ["<add_term>"],
    "<add_term>": ["<add_term> <add> <mul_term>", "<mul_term>"],
    "<mul_term>": ["<mul_term> <mul> <neg_term>", "<neg_term>"],
    "<neg_term>": ["<terminal>", "~ <terminal>"],
    "<terminal>": ["<int>", "(<add_term>)"],
    "<add>": srange("+-"),
    "<mul>": srange("*/"),
    "<int>": ["<digit><int>", "<digit>"],
    "<digit>": srange(string.digits),
}

assert is_valid_grammar(grammar)

Next, we introduce a class to capture test runs' results efficiently. The `TestResult` is an enum with two possible values, `PASS`and `FAIL`. `PASS` donates a passing test case and `FAIL` a failing one.

In [4]:
class TestResult(enum.Enum):
    
    def __repr__(self):
        return self.value
    
    PASS = 'PASS'
    FAIL = 'FAIL'

Now we implement a test function that takes the three arguments of `middle(x, y, z)` and an expected result. This test function compares the return of `middle(x, y, z)` with the desired value and returns `PASS` if they match and `FAIL` otherwise.

In [5]:
def test(function, x, y, z, expected):
    try:
        if function(x, y, z) == expected:
            return TestResult.PASS
        else:
            return TestResult.FAIL
    except BaseException:
        return TestResult.FAIL

def test_middle(x, y, z, expected):
    return test(middle, x, y, z, expected)

In [6]:
source = inspect.getsource(middle)
print(source)

def middle(x, y, z):
    m = z
    if y < z:
        if x < y:
            m = y
        elif x < z:
            m = y 
    else:
        if x > y:
            m = y
        elif x > z:
            m = x # desired line
    return m



In [7]:
middle_py = 'middle.py'
tmp_py = 'tmp.py'

We produced the same results for the test cases, so it seems to work.

### Configuring SFLKit

The `Config` class provides comfortable access to `SFLKit` by defining the fundamental concepts we want to investigate.

We give some information for the config that we need to define. First, we need the path to the source we want to investigate, which we already have in `middle_py`. Next, we need an out, `tmp_py`. We also need:

The language of our subject is `'python'`.
Let's start with `'line'` as the predicates we want to investigate.
We define `'tarantula'` as our evaluation metric for the predicates, i.e., the similarity coefficient.
We also need a list of passing and failing tests used during the analysis.

In [8]:
language='python'
predicates='line'
metrics='Tarantula'
#passing='event-files/0,event-files/1'
#failing='event-files/2'

We define a function that gives as a `Config` object, so we do not need to create it manually every time we change something.

In [9]:
def get_config():
    return Config.create(path=middle_py, working=tmp_py, language=language, predicates=predicates)

Now we can define a function that instruments our subject. We leverage `SFLKit`'s `instrument_config()`, which takes a config we create with our defined `get_config()` and instruments the subject. We can also show the content of the instrumented python file with this function.

In [10]:
def instrument(out=False):
    instrument_config(get_config())

Now we instrument our `middle.py` subject and check the results.

In [11]:
instrument()

sflkit :: INFO     :: I found 11 events in middle.py.
sflkit :: INFO     :: I found 11 events in middle.py.


As you can see, the instrumentation added an import at the beginning to a lib that comes with `SFLKit`, cluing the execution of files together. Moreover, the instrumentation added a function call function of the lib in front of each executable line that tracks the executed lines.

## Get and Analyze Events

Now, we want to extract the events from the execution of tests. Therefore, we need to adjust our test execution function again because the shared library for tracking the events does not know when to start and stop. We need to reset this library before entering our `middle.py` and tell the library to dump the events after the function finishes.

In [12]:
def test_tmp(x, y, z, expected): 
    import tmp
    importlib.reload(tmp)
    tmp.sflkitlib.lib.reset()
    try:
        return test(tmp.middle, x, y, z, expected)
    finally:
        tmp.sflkitlib.lib.dump_events()

We define a path to write the generated event logs.

In [13]:
event_files = 'event-files'

Then, we need a function to generate the event log from the previous test cases. We change the environment variable `EVENTS_PATH` to the output path of the event log file before running each test.

In [14]:
# def run_tests():
#     if os.path.exists(event_files):
#         shutil.rmtree(event_files)
#     os.mkdir(event_files)
#     os.environ['EVENTS_PATH'] = os.path.join(event_files, '0')
#     test_tmp(3, 2, 1, expected=2)
#     os.environ['EVENTS_PATH'] = os.path.join(event_files, '1')
#     test_tmp(3, 1, 2, expected=2)
#     os.environ['EVENTS_PATH'] = os.path.join(event_files, '2')
#     test_tmp(2, 1, 3, expected=2)

With this, we can execute the tests and analyze the result with the help of `analyze_config()` from SFLKit.

In [15]:
# def analyze():
#     run_tests()
#     return analyze_config(get_config())

Let's execute the tests and analyze the event logs for lines and the Tarantula metric.

In [16]:
#results = analyze()

The results look something like this:

In [17]:
#results

This structure maps analysis objects and metrics to a list of sorted suggestions where the fault occurs.

Now, we can put all this together and produce a pretty output that shows us where the fault originates by leveraging `SFLKit`'s `ColorCode` object.

As you can see, the analysis indeed suggests the buggy line 7 as the most suspicious.

But what if lines are not enough to show the fault?

What if the metric we have chosen for evaluation is insufficient?

### Extension using Avicenna
We seek to utilize SFLKit's instrmentation of program code line events to determine constraints describing a specific line's reachability with Avicenna. 

1. We must instrument our test program, middle, and use that output.
2. We access the event files and check what line was called, oracle determines YES or NO.
3. If yes, learn from that event file, make constraint loop with Avicenna
4. If no, call other event files to check for line event YES.


Instrumentation for line events is coded as follows: (event_type, file_name, line_num, event_id)


First we need to instrument our event files for a given program or function.

The oracle has to be able to determine if a given line is called when a test case is run. This means we need to call sflkit's instrumentation for each test case when determining whether it is valuable and can be used for finding constraints that lead to the desired line.

In [18]:
import string
import csv
import logging

from pathlib import Path
#from avicenna import Avicenna
from debugging_framework.input.oracle import OracleResult
from isla.solver import ISLaSolver


We can now go through our event-files folder and iterate through each file, representing a test case. The relevant test cases will be denoted by an OracleResult.BUG for now. We can use this behavior-triggering input as our input for a run of Avicenna to find the constraints leading to this behavior.

The oracle used in this run of Avicenna will be the oracle checking against line-event calls. This requires us to run sflkit using Avicenna's new hypotheses after every run. 

- TODO : update Avicenna and fix up prototype for it 
- TODO : implement prototype functionality using SFLKit and Avicenna 0.2 

#### Testing inputs for regular middle bug. 

In [19]:
middle_grammar = {
    "<start>": ["<stmt>"],
    "<stmt>": ["<x>,<y>,<z>"],
    "<x>": ["<integer>"],
    "<y>": ["<integer>"],
    "<z>": ["<integer>"],
    "<integer>": ["<digit>", "<digit><integer>"],
    "<digit>": [str(num) for num in range(1, 10)]
}


middle_grammar_converted = {
    "<start>": ["<stmt>"],
    "<stmt>": ["str.to.int(<x>),str.to.int(<y>),str.to.int(<z>)"],
    "<x>": ["<integer>"],
    "<y>": ["<integer>"],
    "<z>": ["<integer>"],
    "<integer>": ["<digit>", "<digit><integer>"],
    "<digit>": [str(num) for num in range(1, 10)]
}

In [20]:
# 321, 312, 213
middle_inputs = ['2,3,1', '3,1,2', '2,1,3']

In [21]:
# path = Path('./event-files/')
# if os.path.exists(path):
#     shutil.rmtree(path) 
# # os.mkdir(path) 
# os.environ['EVENTS_PATH'] = os.path.join(path, '0')
# os.environ['EVENTS_PATH'] = os.path.join(path, '1')
# os.environ['EVENTS_PATH'] = os.path.join(path, '2')


In [22]:
middle_py = 'middle.py'
tmp_py = 'tmp.py'
language='python'
predicates='line'
metrics='Tarantula'
#passing='event-files/0,event-files/1'
failing='event-files/2'
def get_config():
    return Config.create(path=middle_py,
                         working=tmp_py,
                         language=language,
                         predicates=predicates,
                         metrics=metrics,
                         failing=failing)
def instrument(out=True):
    instrument_config(get_config())
    if out:
        with open(tmp_py, 'r') as fp:
            print(fp.read())

In [23]:
instrument()

sflkit :: INFO     :: I found 11 events in middle.py.
sflkit :: INFO     :: I found 11 events in middle.py.


import sflkitlib.lib

def middle(x, y, z):
    sflkitlib.lib.add_line_event(0)
    m = z
    sflkitlib.lib.add_line_event(1)
    if y < z:
        sflkitlib.lib.add_line_event(2)
        if x < y:
            sflkitlib.lib.add_line_event(3)
            m = y
        else:
            sflkitlib.lib.add_line_event(4)
            if x < z:
                sflkitlib.lib.add_line_event(5)
                m = y
    else:
        sflkitlib.lib.add_line_event(6)
        if x > y:
            sflkitlib.lib.add_line_event(7)
            m = y
        else:
            sflkitlib.lib.add_line_event(8)
            if x > z:
                sflkitlib.lib.add_line_event(9)
                m = x
    sflkitlib.lib.add_line_event(10)
    return m


In [24]:
def analyzer_conf(conf: Config, factory):
    analyzer = Analyzer(irrelevant_event_files=conf.failing, 
                        relevant_event_files=conf.passing,
                        factory=factory)
    return analyzer

In [25]:
factory = factory.LineFactory()
line_event = analysis_type.AnalysisType(0)
line_event

LINE

In [26]:
analyzer = analyzer_conf(get_config(), factory=factory)

In [27]:
# results = analyzer.analyze()
# results

# analyzer needs event files 
analyzer.analyze()

In [28]:
coverage: List[Line] = analyzer.get_coverage()
coverage = {line.line for line in coverage}
print(coverage)

{2, 3, 9, 10, 13}


In [29]:
def middle_inp_conv(inp):
    inp = inp.__str__()
    middle_input = inp.split(',')
    
    converted_inp = [
        int(middle_input[0]),
        int(middle_input[1]),
        int(middle_input[2])
    ]
    
    return converted_inp

In [30]:
# from typing import List

# from sflkit.analysis.analysis_type import AnalysisType
# from sflkit.analysis.spectra import Line

# analyzer2 = analyzer.analyze()
# print("hi")
# coverage: List[Line] = analyzer2.get_coverage(AnalysisType.LINE)
# coverage = {line.line for line in coverage}
# print(coverage)
from avicenna.avix import *
from avicenna.oracle_construction import * 
avix_oracle = construct_oracle(
                            program_under_test='middle',
                            inp_converter=middle_inp_conv,
                            timeout=10,
                            line = 7,
                            resource_path='rsc/',
                            program_oracle= None,
                            )


In [31]:
avix_oracle('2,1,3')

<OracleResult.FAILING: 'FAILING'>

In [32]:
def middle_oracle(x, y, z):
    # convert inputs
    
    sorted_inputs = [x, y, z]
    sorted_inputs.sort()
    return sorted_inputs[1]
    
    inputs = input.split()
    result_list = [x, y, z]
    result_list.sort()
    return result_list[1]

In [33]:
regular_oracle = construct_oracle(
                            program_under_test=middle,
                            timeout=10,
                            program_oracle=middle_oracle,
                            default_oracle_result=OracleResult.PASSING)



In [34]:
param = list(map(int, str('2,1,3').strip().split(',')))
middle_inputs

['2,3,1', '3,1,2', '2,1,3']

In [35]:
regular_oracle('3,1,2')


<OracleResult.PASSING: 'PASSING'>

In [36]:
avicenna = Avicenna(
    grammar=middle_grammar,
    initial_inputs=middle_inputs,
    oracle=avix_oracle,
    max_iterations=20,
    top_n_relevant_features=3,
)

In [37]:
avicenna2 = Avicenna(
    grammar=middle_grammar,
    initial_inputs=middle_inputs,
    oracle=regular_oracle,
    max_iterations=10,
    top_n_relevant_features=3, 
)

In [38]:
import warnings
# Suppress the specific SHAP warning
warnings.filterwarnings(
    "ignore",
    message="LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray",
)
warnings.filterwarnings(
    "ignore", 
    message="No further splits with positive gain, best gain: -inf"
)

In [39]:
logging.basicConfig(filename='avicenna.log', filemode='w', encoding='utf-8', level=logging.INFO, force=True)
# only 2 constraints used in the end why
best_invariant = avicenna.explain() # unparse with islaunparse for further use


KeyboardInterrupt: 

In [ ]:
another_invariant = avicenna2.explain() # unparse with islaunparse for further use

{}
[(ConjunctiveFormula(ForallFormula(BoundVariable("elem_1", "<x>"), Constant("start", "<start>"), ExistsFormula(BoundVariable("elem_2", "<y>"), Constant("start", "<start>"), SMTFormula('(> (str.to_int elem_1) (str.to_int elem_2))', BoundVariable("elem_1", "<x>"), BoundVariable("elem_2", "<y>"), ))), ForallFormula(BoundVariable("elem_0", "<z>"), Constant("start", "<start>"), ExistsFormula(BoundVariable("elem_3", "<x>"), Constant("start", "<start>"), SMTFormula('(> (str.to_int elem_0) (str.to_int elem_3))', BoundVariable("elem_0", "<z>"), BoundVariable("elem_3", "<x>"), )))), 1.0, 1.0), (ConjunctiveFormula(ForallFormula(BoundVariable("elem_1", "<x>"), Constant("start", "<start>"), ExistsFormula(BoundVariable("elem_2", "<y>"), Constant("start", "<start>"), SMTFormula('(> (str.to_int elem_1) (str.to_int elem_2))', BoundVariable("elem_1", "<x>"), BoundVariable("elem_2", "<y>"), ))), ForallFormula(BoundVariable("elem_0", "<z>"), Constant("start", "<start>"), ExistsFormula(BoundVariable("el

In [ ]:
print(another_invariant)

(ConjunctiveFormula(ForallFormula(BoundVariable("elem_1", "<x>"), Constant("start", "<start>"), ExistsFormula(BoundVariable("elem_2", "<y>"), Constant("start", "<start>"), SMTFormula('(> (str.to_int elem_1) (str.to_int elem_2))', BoundVariable("elem_1", "<x>"), BoundVariable("elem_2", "<y>"), ))), ForallFormula(BoundVariable("elem_0", "<z>"), Constant("start", "<start>"), ExistsFormula(BoundVariable("elem_3", "<x>"), Constant("start", "<start>"), SMTFormula('(> (str.to_int elem_0) (str.to_int elem_3))', BoundVariable("elem_0", "<z>"), BoundVariable("elem_3", "<x>"), )))), 1.0, 1.0)


In [ ]:
print(best_invariant)
print(best_invariant[1])

(ConjunctiveFormula(ForallFormula(BoundVariable("elem_1", "<z>"), Constant("start", "<start>"), ExistsFormula(BoundVariable("elem_2", "<y>"), Constant("start", "<start>"), SMTFormula('(> (str.to_int elem_1) (str.to_int elem_2))', BoundVariable("elem_1", "<z>"), BoundVariable("elem_2", "<y>"), ))), ForallFormula(BoundVariable("elem_0", "<z>"), Constant("start", "<start>"), ExistsFormula(BoundVariable("elem_3", "<x>"), Constant("start", "<start>"), SMTFormula('(> (str.to_int elem_0) (str.to_int elem_3))', BoundVariable("elem_0", "<z>"), BoundVariable("elem_3", "<x>"), )))), 0.8269581056466302, 1.0)
0.8269581056466302


In [ ]:
solver1 = ISLaSolver(
    grammar=middle_grammar,
    formula=another_invariant[0],
    enable_optimized_z3_queries=False,
)

solver2 = ISLaSolver(
    grammar=middle_grammar,
    formula=best_invariant[0],
    enable_optimized_z3_queries=False,
)

results1 = []
results2 = []

# print(best_invariant[0])
# print(another_invariant[0])
for _ in range(1,10):
    results1.append(solver1.solve())
    
print("stop")
#    avix_oracle()
for _ in range(1,10):
    results2.append(solver2.solve())

stop


In [ ]:
for input in results1:
    if avix_oracle(input) == OracleResult.FAILING:
        print(input)
        print(avix_oracle(input))

3,1,8
FAILING
3,1,9
FAILING
3,1,4
FAILING
3,1,5
FAILING
3,1,6
FAILING
3,1,7
FAILING
3,1,18
FAILING
3,1,14
FAILING
3,1,11
FAILING


In [ ]:
for input in results2:
    if avix_oracle(input) == OracleResult.FAILING:
        print(input)
        print(avix_oracle(input))
for input in results2:
    print(input)

2,2,3
FAILING
1,1,2
FAILING
4,4,7
FAILING
6,4,7
FAILING
5,4,7
FAILING
1,2,3
2,2,3
1,1,2
2,4,7
4,4,7
1,4,7
6,4,7
3,4,7
5,4,7


In [ ]:
solver = ISLaSolver(
    grammar=middle_grammar,
    formula=best_invariant[0],
    enable_optimized_z3_queries=False,
)

# this isnt working rn why not raaaaaaaaaaaaaaaaaaaaaaa
# should be inputs of type 2, 3, 1
for _ in range(1,10):
    print(solver.solve())


4,1,5
2,1,5
1,1,5
3,1,5
4,1,6
2,1,6
3,1,6
5,1,6
1,1,6


In [ ]:
# call func for middle, converts string input to usable integer values

def call_func_middle(inp: str):
    
    inp = inp.__str__()
    
    middle_input = inp.split(',')
    converted_inp =  [int(middle_input[0]), int(middle_input[1]), int(middle_input[2])]
    return converted_inp

In [ ]:
test_path = Path('rsc/')
print(Path.exists(test_path))


True


In [ ]:
from avicenna.avix import *
from avicenna.oracle_construction import * 

In [ ]:
def middle_inp_conv(inp):
    inp = inp.__str__()
    middle_input = inp.split(',')
    
    converted_inp = [
        int(middle_input[0]),
        int(middle_input[1]),
        int(middle_input[2])
    ]
    
    return converted_inp

In [ ]:
import tmp
importlib.reload(tmp)
tmp.sflkitlib.lib.reset()
avix_oracle = construct_oracle(
                            program_under_test='middle',
                            inp_converter=middle_inp_conv,
                            timeout=10,
                            line = 7,
                            resource_path='rsc/',
                            program_oracle= None)


In [ ]:
avix_oracle("6,3,7")

<OracleResult.FAILING: 'FAILING'>

In [ ]:
# # double check if this works

# AviX.create_event_file(instrumented_function='middle',
#                        #instr_path='tmp',
#                        inp = '2,1,3',
#                        conversion_func=middle_inp_conv,
#                        event_path='rsc/event_file'
#                        )

In [ ]:
avix = AviX(grammar=middle_grammar,
            initial_inputs=middle_inputs,
            oracle=avix_oracle,
            max_iterations=10,
            desired_line=7,
            put_path='middle.py',
            #instr_path='instrumented.py',
            min_precision = 0.7,)

In [ ]:
from fuzzingbook.GrammarFuzzer import GrammarFuzzer, DerivationTree

middle_grammar_converted = {
    "<start>": ["<stmt>"],
    "<stmt>": ["str.to.int(<x>),str.to.int(<y>),str.to.int(<z>)"],
    "<x>": ["<integer>"],
    "<y>": ["<integer>"],
    "<z>": ["<integer>"],
    "<integer>": ["<digit>", "<digit><integer>"],
    "<digit>": [str(num) for num in range(1, 10)]
}

fuzzer = GrammarFuzzer(middle_grammar)

for i in range(10):
    print(fuzzer.fuzz())

14,977,74
9,1,1
86,87619,98
75,88,2
4,3,7
4,8,74
67,11,97
6,25,182
7958,6,82
5,46,2


https://github.com/uds-se/sflkit

<img src="qrcode.png" style="width:500px">